# Image Matching using SURF

In [4]:
(ns origami.surf-flann-matching-test 
  (:require [opencv4.core :refer :all] [opencv4.utils :as u])
  (:import [java.util ArrayList]
           [org.opencv.core MatOfKeyPoint MatOfDMatch MatOfByte DMatch Mat Scalar]
           [org.opencv.features2d Features2d DescriptorMatcher]
           [org.opencv.highgui HighGui]
           [org.opencv.imgcodecs Imgcodecs]
           [org.opencv.xfeatures2d SURF]))

nil

In [5]:
(defn filtered-matches [knn-matches]
  (let [ratio-thresh 0.7
        good-matches (MatOfDMatch.)]
    (.fromList good-matches
               (->> knn-matches
                    (filter #(> (.rows %) 1))
                    (map #(.toArray %))
                    (filter #(< (.distance (first %))
                                (* ratio-thresh (.distance (second %)))))
                    (map #(first %))))
    good-matches))

(defn image-matches [object-image scene-image]
  (let [hessian-threshold 400
        n-octaves 4
        n-octave-layers 3
        extended false
        upright false
        detector (SURF/create hessian-threshold n-octaves n-octave-layers extended upright)
        keypoints1 (MatOfKeyPoint.)
        keypoints2 (MatOfKeyPoint.)
        descriptors1 (Mat.)
        descriptors2 (Mat.)
        matcher (DescriptorMatcher/create DescriptorMatcher/FLANNBASED)
        knn-matches (ArrayList.)
        ratio-thresh 0.7
        image-matches (Mat.)]
    (.detectAndCompute detector object-image (Mat.) keypoints1 descriptors1)
    (.detectAndCompute detector scene-image (Mat.) keypoints2 descriptors2)
    (.knnMatch matcher descriptors1 descriptors2 knn-matches 2)
    (Features2d/drawMatches object-image keypoints1
                            scene-image keypoints2
                            (filtered-matches knn-matches) image-matches
                            (Scalar/all -1) (Scalar/all -1) (MatOfByte.)
                            Features2d/DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
   image-matches))

#'origami.surf-flann-matching-test/image-matches

In [6]:
(let [object-image (imread "resources/box.png" Imgcodecs/IMREAD_GRAYSCALE)
      scene-image (imread "resources/box_in_scene.png" Imgcodecs/IMREAD_GRAYSCALE)
      image-matches (image-matches object-image scene-image)]
  (u/>>> image-matches))

UnsatisfiedLinkError org.opencv.xfeatures2d.SURF.create_0(DIIZZ)J  org.opencv.xfeatures2d.SURF.create_0 (SURF.java:-2)


class java.lang.UnsatisfiedLinkError: 